In [ ]:
#pip install mediapipe opencv-python

In [1]:
import mediapipe as mp

In [2]:
import pandas as pd
import cv2

In [11]:
video_path = "nofall_new.avi"
cap = cv2.VideoCapture(video_path)

In [12]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)

In [13]:
output_txt = open("nofall_new.txt", "w")

frame_id = 0
def get_coords(landmarks, idx):
    pt = landmarks[idx]
    return (pt.x, pt.y) if pt.visibility > 0.5 else (None, None)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)

    if result.pose_landmarks:
        lm = result.pose_landmarks.landmark

        # Get torso
        l_sh, r_sh = get_coords(lm, 11), get_coords(lm, 12)
        l_hip, r_hip = get_coords(lm, 23), get_coords(lm, 24)
        # Legs
        l_knee, r_knee = get_coords(lm, 25), get_coords(lm, 26)
        l_ankle, r_ankle = get_coords(lm, 27), get_coords(lm, 28)

        if None not in l_sh + r_sh + l_hip + r_hip + l_knee + r_knee + l_ankle + r_ankle:
            mid_sh = [(l_sh[0] + r_sh[0]) / 2, (l_sh[1] + r_sh[1]) / 2]
            mid_hip = [(l_hip[0] + r_hip[0]) / 2, (l_hip[1] + r_hip[1]) / 2]

            line = f"{frame_id} {mid_sh[0]} {mid_sh[1]} {mid_hip[0]} {mid_hip[1]} " \
                   f"{l_hip[0]} {l_hip[1]} {r_hip[0]} {r_hip[1]} " \
                   f"{l_hip[0]} {l_hip[1]} {l_knee[0]} {l_knee[1]} {l_ankle[0]} {l_ankle[1]} " \
                   f"{r_hip[0]} {r_hip[1]} {r_knee[0]} {r_knee[1]} {r_ankle[0]} {r_ankle[1]}\n"
            output_txt.write(line)

    frame_id += 1

cap.release()
output_txt.close()
pose.close()
print("Saved as skeleton_lines.txt")


Saved as skeleton_lines.txt


In [10]:
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video writer
out = cv2.VideoWriter('annotated_output.avi',
                      cv2.VideoWriter_fourcc(*'XVID'),
                      fps, (frame_width, frame_height))

with open("skeleton_lines.txt", "r") as f:
    lines = f.readlines()

def denorm(x, y, w, h):
    return int(x * w), int(y * h)

for i, line in enumerate(lines):
    ret, frame = cap.read()
    if not ret:
        break

    parts = list(map(float, line.strip().split()))
    _, bx1, by1, bx2, by2, \
    lx1, ly1, rx1, ry1, \
    llh_x, llh_y, llk_x, llk_y, lla_x, lla_y, \
    rlh_x, rlh_y, rlk_x, rlk_y, rla_x, rla_y = parts

    h, w, _ = frame.shape

    # Body
    cv2.line(frame, denorm(bx1, by1, w, h), denorm(bx2, by2, w, h), (255, 0, 0), 2)     # torso (blue)
    cv2.line(frame, denorm(lx1, ly1, w, h), denorm(rx1, ry1, w, h), (0, 255, 0), 2)     # pelvis (green)

    # Left leg
    cv2.line(frame, denorm(llh_x, llh_y, w, h), denorm(llk_x, llk_y, w, h), (0, 255, 255), 2)
    cv2.line(frame, denorm(llk_x, llk_y, w, h), denorm(lla_x, lla_y, w, h), (0, 255, 255), 2)

    # Right leg
    cv2.line(frame, denorm(rlh_x, rlh_y, w, h), denorm(rlk_x, rlk_y, w, h), (255, 255, 0), 2)
    cv2.line(frame, denorm(rlk_x, rlk_y, w, h), denorm(rla_x, rla_y, w, h), (255, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print("🎥 Saved annotated_output.avi")

🎥 Saved annotated_output.avi
